# Uncommon Rise and Short Position Strategy Analysis

This analysis explores how uncommon rises in the NASDAQ index (symbol: `NQ=F`) could signal profitable short positions. The analysis considers the behavior of the 5-day percentage change in adjusted closing prices and identifies the points at which the percentage change exceeds a defined threshold. The strategy aims to enter short positions following these uncommon rises and exit when the market drops significantly. 

## Objectives:
- Identify uncommon rises based on a threshold.
- Simulate short positions by entering after a rise and exiting at the most profitable drop.
- Calculate key statistics to evaluate the effectiveness of the strategy.

## Data Collection

The data used for this analysis was downloaded from Yahoo Finance using the `yfinance` library. The dataset includes adjusted closing prices of the NASDAQ index from January 1, 2015, to the present day.


In [3]:
# Import Libraly
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# Download data for NASDAQ
data = yf.download('NQ=F', start='2015-01-01', end=datetime.now())

# Displaying the first few rows of data
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,4240.25,4272.75,4201.00,4214.25,4214.25,229555
2015-01-05,4216.00,4220.75,4145.00,4161.75,4161.75,313771
2015-01-06,4166.50,4172.50,4082.00,4102.25,4102.25,426154
2015-01-07,4105.50,4163.25,4102.75,4151.50,4151.50,328184
2015-01-08,4152.25,4242.75,4152.25,4232.25,4232.25,272056


## Data Preprocessing

We calculated the 1-day and 5-day percentage changes in the adjusted closing prices to identify unusual price movements. These calculations were done as follows:

- **1-Day Percentage Change**: The daily percentage change compared to the previous day's adjusted close.
- **5-Day Percentage Change**: The percentage change compared to 5 days ago.

The next step involved dropping any rows with missing values.

In [6]:
# Calculate the 1-day and 5-day percentage changes for Adj Close
df = data.copy()
df['Pct Change Close 1D'] = ((df['Adj Close'] / df['Adj Close'].shift(1)) - 1) * 100
df['Pct Change Close 5D'] = ((df['Adj Close'] / df['Adj Close'].shift(5)) - 1) * 100

# Drop rows with missing values
df.dropna(inplace=True)

# Displaying the first few rows of data
df.head()

,Open,High,Low,Close,Adj Close,Volume,Pct Change Close 1D,Pct Change Close 5D
Date,,,,,,,,
2015-01-09,4232.25,4252.50,4181.00,4200.75,4200.75,341966,-0.744285,-0.320342
2015-01-12,4201.25,4227.50,4151.00,4166.50,4166.50,319438,-0.815331,0.114135
2015-01-13,4169.50,4247.75,4126.75,4158.50,4158.50,476645,-0.192008,1.371199
2015-01-14,4158.00,4169.25,4100.50,4145.50,4145.50,450812,-0.312613,-0.144526
2015-01-15,4147.50,4190.00,4066.00,4089.00,4089.00,456035,-1.362924,-3.384724


### Identifying Uncommon Rises

To identify the significant jumps in the market, we defined a threshold based on the mean and standard deviation of the 5-day percentage changes. Any day where the 5-day percentage change exceeded this threshold was considered a signal of an uncommon rise.

The threshold was calculated as:

$$
\text{Threshold} = \mu_{\text{5-day Pct Change}} + 2 \times \sigma_{\text{5-day Pct Change}}
$$

Where:
- $\mu_{\text{5-day Pct Change}}$  is the mean of the 5-day percentage change.
- $\sigma_{\text{5-day Pct Change}}$  is the standard deviation of the 5-day percentage change.


In [23]:
# Calculate the mean and standard deviation of the 5-day percentage change
mean_pct_change_5D = df['Pct Change Close 5D'].mean()
std_pct_change_5D = df['Pct Change Close 5D'].std()

# Define the threshold for a significant jump
threshold_close_5D = mean_pct_change_5D + 2 * std_pct_change_5D

# Flag significant jumps in returns based on the 5-day percentage change
df_jump = df.loc[df['Pct Change Close 5D'] > threshold_close_5D]

# Display the first few rows where significant jumps were detected
df_jump.head()  

,Open,High,Low,Close,Adj Close,Volume,Pct Change Close 1D,Pct Change Close 5D
Date,,,,,,,,
2015-08-31,4324.00,4330.25,4258.50,4271.75,4271.75,303204,-1.413570,6.707051
2015-10-28,4635.25,4678.25,4600.50,4673.75,4673.75,306875,0.895893,6.239700
2016-02-17,4089.75,4211.50,4068.25,4196.75,4196.75,291649,2.541079,6.401724
2018-02-15,6681.00,6827.50,6669.75,6816.00,6816.00,458423,1.929116,7.895049
2018-11-30,6902.25,6967.25,6871.50,6949.50,6949.50,485575,0.524355,6.407901


### Filtering the Notice Dates

Once the uncommon rises are identified based on the threshold, the next step is to filter out consecutive signals and only retain those that occur at least one day apart. This avoids considering redundant signals for consecutive days, which could skew the analysis.

In [36]:
# Notice Date
notice_date_list = []
for i in range(len(df_jump)):
    notice_date = df_jump.index[i]
    notice_date_list.append(df.index.get_loc(notice_date))

# Corrected the Notice Date
filtered_notice_dates = []
for i in range(len(notice_date_list)):
    if i == 0 or (notice_date_list[i] - notice_date_list[i - 1] > 1):
        filtered_notice_dates.append(notice_date_list[i])

## Simulating Short Positions

For each detected uncommon rise, we simulated entering a short position the day after the rise. The short position was exited based on the most significant drop in the following 10 days. The key factors tracked were:
- **Short entry date**: The day after the rise.
- **Short exit date**: The day with the most significant drop within 10 days.
- **Max drop**: The maximum percentage loss during the short position.

The strategy was then evaluated using the following metrics:
- **Days to short entry**: The number of days after the uncommon rise before entering a short position.
- **Days to short exit**: The number of days after the short entry to the exit date.
- **Max drop**: The maximum drop in price during the short position.

In [39]:
# Create an empty dictionary to store the short position entry/exit info
short_positions = {}

# Lists to store values for averaging later
days_to_entry_list = []
days_to_exit_list = []
max_drop_list = []

# Loop through each uncommon rise date and simulate short positions
for notice_date in filtered_notice_dates:
    # Initialize variables to track the (short entry)
    start_drop_date = None
    days_after_notice = 0
    
    # Loop through the rows starting from the day after the notice date
    for i in range(notice_date + 1, len(df)):
        # If the percentage change shifts from positive to negative (short entry)
        if df['Pct Change Close 1D'].iloc[i] < 0 and df['Pct Change Close 1D'].iloc[i-1] > 0:
            start_drop_date = i
            days_after_notice = i - notice_date
            break

    # If a drop was found, calculate the returns to find the short exit
    if start_drop_date is not None:
        min_return = 0
        exit_day = None
        
        # Calculate the returns for up to 10 days after the start drop date
        for j in range(1, 11):
            if start_drop_date + j < len(df):
                # Calculate percentage return from the drop start date to day j
                return_j = (df['Adj Close'].iloc[start_drop_date + j] / df['Adj Close'].iloc[start_drop_date] - 1) * 100
                
                # Track the day with the most profitable for the short position
                if return_j < min_return:
                    min_return = return_j
                    exit_day = start_drop_date + j
        
        # Save the entry and exit points along with the return 
        if exit_day is not None:
            entry_date = df.index[start_drop_date]  # Short entry date
            exit_date = df.index[exit_day]  # Short exit date
            days_to_exit = exit_day - start_drop_date 
            
            # Store short position details
            short_positions[df.index[notice_date]] = (entry_date, exit_date, days_after_notice, days_to_exit, min_return)

            # Document info for later analysis
            days_to_entry_list.append(days_after_notice)
            days_to_exit_list.append(days_to_exit)
            max_drop_list.append(min_return)

## Results and Insights

The strategy identifies several potential short positions based on significant market rises. Here are the key results:

- **Notice of uncommon rise**: The date of the significant rise.
- **Short entry**: The day the short position is entered.
- **Short exit**: The day the short position is exited.
- **Max drop**: The maximum price drop during the short position.

### Key Metrics:
- **Average days to short entry**: The average time between the uncommon rise and entering the short position.
- **Average days to short exit**: The average time to exit the short position after entry.
- **Average max drop**: The average percentage drop during the short position.

In [46]:
# Output of the results
for notice_date, (entry_date, exit_date, days_to_entry, days_to_exit, min_return) in short_positions.items():
    print(f"Notice of uncommon rise: {notice_date}")
    print(f"  -> Short entry: {entry_date} ({days_to_entry} days after notice)")
    print(f"  -> Short exit: {exit_date} ({days_to_exit} days after short entry)")
    print(f"  -> Max drop: {min_return:.2f}%\n")

# Calculate and Display the Averages
average_days_to_entry = sum(days_to_entry_list) / len(days_to_entry_list) if days_to_entry_list else 0
average_days_to_exit = sum(days_to_exit_list) / len(days_to_exit_list) if days_to_exit_list else 0
average_max_drop = sum(max_drop_list) / len(max_drop_list) if max_drop_list else 0

print(f"Average days to short entry: {average_days_to_entry:.2f}")
print(f"Average days to short exit: {average_days_to_exit:.2f}")
print(f"Average max drop: {average_max_drop:.2f}%")

Notice of uncommon rise: 2015-08-31 00:00:00
  -> Short entry: 2015-09-03 00:00:00 (3 days after notice)
  -> Short exit: 2015-09-04 00:00:00 (1 days after short entry)
  -> Max drop: -0.84%

Notice of uncommon rise: 2015-10-28 00:00:00
  -> Short entry: 2015-10-29 00:00:00 (1 days after notice)
  -> Short exit: 2015-11-12 00:00:00 (10 days after short entry)
  -> Max drop: -1.64%

Notice of uncommon rise: 2018-02-15 00:00:00
  -> Short entry: 2018-02-16 00:00:00 (1 days after notice)
  -> Short exit: 2018-02-21 00:00:00 (2 days after short entry)
  -> Max drop: -0.39%

Notice of uncommon rise: 2018-11-30 00:00:00
  -> Short entry: 2018-12-04 00:00:00 (2 days after notice)
  -> Short exit: 2018-12-19 00:00:00 (10 days after short entry)
  -> Max drop: -6.89%

Notice of uncommon rise: 2019-01-10 00:00:00
  -> Short entry: 2019-01-11 00:00:00 (1 days after notice)
  -> Short exit: 2019-01-14 00:00:00 (1 days after short entry)
  -> Max drop: -1.02%

Notice of uncommon rise: 2020-03-26 00

## Conclusion

The analysis indicates that entering short positions following uncommon rises in the NASDAQ index can be a profitable strategy. Several instances have shown significant price drops within days of entering a short position, supporting the effectiveness of this strategy.

### Insights:
- On average, it takes **2.12 days** to enter a short position.
- On average, it takes **5.12 days** to exit a short position.
- The average maximum drop in price during this period is **-3.18%**.

This suggests that the strategy of shorting after uncommon rises is effective in capturing downward price movements over a relatively short time frame.